# Outcomes data

In [7]:
import pandas as pd
import numpy as np

outcomes_df = pd.read_csv("../data/aac_outcomes.csv")
outcomes_df.drop(['date_of_birth','monthyear','outcome_subtype'], axis = 1, inplace = True)
outcomes_df = outcomes_df[outcomes_df['animal_type'].isin(['Cat', 'Dog'])]
# Reset index after filtering
outcomes_df = outcomes_df.reset_index(drop=True)

# print(outcomes_df)
print(outcomes_df.shape)

(163904, 9)


In [8]:
outcomes_df.columns # We will drop animal_id later + datetime as well as it leaks data + sex_upon_outcome and age_upon_outcome as well since that datais not available at the intake where the shelter wants to predict

Index(['animal_id', 'name', 'outcome_type', 'animal_type', 'sex_upon_outcome',
       'age_upon_outcome', 'breed', 'color', 'datetime'],
      dtype='object')

In [9]:
outcomes_df.isnull().sum()

animal_id               0
name                41759
outcome_type           39
animal_type             0
sex_upon_outcome        1
age_upon_outcome        5
breed                   0
color                   0
datetime                0
dtype: int64

In [10]:
# outcomes_df['animal_id'].value_counts() # We will keep the latest outcome one

In [11]:
# outcomes_df['name'].nunique()

In [12]:
# outcomes_df['outcome_type'].value_counts()

In [13]:
# outcomes_df['animal_type'].value_counts()

In [14]:
# outcomes_df['sex_upon_outcome'].value_counts()

In [15]:
# outcomes_df['age_upon_outcome'].value_counts()

In [16]:
# outcomes_df['breed'].nunique()

In [17]:
# outcomes_df['color'].nunique()

In [18]:
# outcomes_df['datetime'] # We need to convert this to datetime object and the format should be consistent accross the data

In [19]:
import pandas as pd
import dateutil

def parse_any_datetime(val, default_tz="UTC"):
    try:
        dt = pd.to_datetime(val, utc=True)   # works for both offset and normal
        return dt
    except Exception:
        # Fallback: parse with dateutil and assign default timezone
        dt = dateutil.parser.parse(val)
        return dt.tz_localize(default_tz).tz_convert("UTC")

outcomes_df['datetime'] = outcomes_df['datetime'].astype(str).apply(parse_any_datetime)
outcomes_df['datetime'] = outcomes_df['datetime'].dt.tz_localize(None)

In [20]:
# outcomes_df['animal_id'].value_counts()

In [21]:
# outcomes_df[outcomes_df['animal_id'] == 'A721033']

In [22]:
# Step 1: Count number of visits per animal_id
visit_counts = outcomes_df['animal_id'].value_counts()

# Step 2: Keep only the latest record per animal_id
outcomes_latest = outcomes_df.sort_values(['animal_id', 'datetime']).groupby('animal_id').tail(1)

# Step 3: Add visit_count column
outcomes_latest['visit_count'] = outcomes_latest['animal_id'].map(visit_counts)

# Step 4: Reset index for neatness
outcomes_latest = outcomes_latest.reset_index(drop=True)

In [23]:
outcomes_latest[outcomes_latest['animal_id'] == 'A721033']

,animal_id,name,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,datetime,visit_count
40454,A721033,Lil Bit,Rto-Adopt,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle,2019-10-20 11:35:00,33


In [24]:
outcomes_latest['animal_id'].value_counts()

animal_id
A929710    1
A006100    1
A047759    1
A134067    1
A141142    1
          ..
A212672    1
A210457    1
A208755    1
A200922    1
A197810    1
Name: count, Length: 146409, dtype: int64

In [25]:
outcomes_latest['datetime']

0        2017-12-07 05:00:00
1        2014-04-07 15:12:00
2        2013-11-16 11:54:00
3        2013-11-17 11:40:00
4        2014-11-14 19:28:00
                 ...        
146404   2025-05-02 18:44:00
146405   2025-05-02 18:43:00
146406   2025-05-03 13:19:00
146407   2025-05-03 11:27:00
146408   2025-05-04 17:13:03
Name: datetime, Length: 146409, dtype: datetime64[ns]

In [26]:
outcomes_latest.columns 

Index(['animal_id', 'name', 'outcome_type', 'animal_type', 'sex_upon_outcome',
       'age_upon_outcome', 'breed', 'color', 'datetime', 'visit_count'],
      dtype='object')

In [27]:
outcomes_latest

,animal_id,name,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,datetime,visit_count
0,A006100,Scamp,Return to Owner,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,2017-12-07 05:00:00,3
1,A047759,Oreo,Transfer,Dog,Neutered Male,10 years,Dachshund,Tricolor,2014-04-07 15:12:00,1
2,A134067,Bandit,Return to Owner,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White,2013-11-16 11:54:00,1
3,A141142,Bettie,Return to Owner,Dog,Spayed Female,15 years,Labrador Retriever/Pit Bull,Black/White,2013-11-17 11:40:00,1
4,A163459,Sasha,Return to Owner,Dog,Intact Female,15 years,Miniature Schnauzer Mix,Black/Gray,2014-11-14 19:28:00,1
...,...,...,...,...,...,...,...,...,...,...
146404,A929599,NaN,Transfer,Cat,Unknown,4 days,Domestic Shorthair,Brown Tabby,2025-05-02 18:44:00,1
146405,A929600,NaN,Transfer,Cat,Unknown,4 days,Domestic Shorthair,Brown Tabby,2025-05-02 18:43:00,1
146406,A929602,NaN,Transfer,Cat,Unknown,1 year,Domestic Shorthair,Black,2025-05-03 13:19:00,1
146407,A929631,Fiona,Euthanasia,Dog,Spayed Female,16 years,Chihuahua Shorthair/Dachshund,Tan,2025-05-03 11:27:00,1


# Intakes data

In [28]:
import pandas as pd
import numpy as np
intakes_df = pd.read_csv("../data/aac_intakes.csv")

In [29]:
intakes_df.drop(['datetime2','found_location'], axis = 1, inplace = True)
intakes_df = intakes_df[intakes_df['animal_type'].isin(['Cat', 'Dog'])]
# Reset index after filtering
intakes_df = intakes_df.reset_index(drop=True)

print(intakes_df)
print(intakes_df.shape)
print(intakes_df.columns)

                       datetime animal_id     name      intake_type  \
0       2013-10-01T16:10:00.000   A664308    Bruce            Stray   
1       2013-10-11T18:13:00.000   A664968    Simba            Stray   
2       2013-10-14T11:23:00.000   A665140      NaN            Stray   
3       2013-10-01T15:58:00.000   A659642  *Trevor  Owner Surrender   
4       2013-10-04T11:05:00.000   A664489   Nikole            Stray   
...                         ...       ...      ...              ...   
163927  2025-04-18T08:44:00.000   A928711      Red            Stray   
163928  2025-04-29T14:05:00.000   A929392      NaN            Stray   
163929  2025-04-15T14:13:00.000   A928558  *Murphy            Stray   
163930  2025-05-02T22:01:00.000   A929633      NaN            Stray   
163931  2025-04-23T16:53:00.000   A926662   Pancho    Public Assist   

       intake_condition animal_type sex_upon_intake age_upon_intake  \
0                Normal         Dog     Intact Male         6 years   
1    

In [30]:
# intakes_df['animal_id'].value_counts()

In [31]:
intakes_df['datetime'] = intakes_df['datetime'].astype(str).apply(parse_any_datetime)
intakes_df['datetime'] = intakes_df['datetime'].dt.tz_localize(None)

In [32]:
# intakes_df[intakes_df['animal_id'] == 'A721033']

In [33]:
# Keep only the latest outcome for each animal_id
intakes_df = intakes_df.sort_values('datetime')  # sort by datetime
intakes_df = intakes_df.groupby('animal_id').tail(1)  # keep last row per animal_id

# Reset index for neatness
intakes_latest = intakes_df.reset_index(drop=True)

print(intakes_latest.shape)
print(intakes_latest.head())

(146450, 10)
             datetime animal_id    name intake_type intake_condition  \
0 2013-10-01 07:51:00   A521520    Nina       Stray           Normal   
1 2013-10-01 08:33:00   A664237     NaN       Stray           Normal   
2 2013-10-01 08:33:00   A664236     NaN       Stray           Normal   
3 2013-10-01 08:33:00   A664235     NaN       Stray           Normal   
4 2013-10-01 08:53:00   A664233  Stevie       Stray          Injured   

  animal_type sex_upon_intake age_upon_intake                         breed  \
0         Dog   Spayed Female         7 years  Border Terrier/Border Collie   
1         Cat         Unknown          1 week        Domestic Shorthair Mix   
2         Cat         Unknown          1 week        Domestic Shorthair Mix   
3         Cat         Unknown          1 week        Domestic Shorthair Mix   
4         Dog   Intact Female         3 years                  Pit Bull Mix   

          color  
0     White/Tan  
1  Orange/White  
2  Orange/White  
3  Oran

In [34]:
intakes_latest[intakes_latest['animal_id'] == 'A721033']

,datetime,animal_id,name,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
90312,2019-10-05 12:31:00,A721033,Lil Bit,Public Assist,Normal,Dog,Neutered Male,4 years,Rat Terrier Mix,Tricolor/Brown Brindle


In [35]:
# intakes_latest['animal_id'].value_counts()

In [36]:
# intakes_latest['datetime']

In [37]:
outcomes_latest.columns

Index(['animal_id', 'name', 'outcome_type', 'animal_type', 'sex_upon_outcome',
       'age_upon_outcome', 'breed', 'color', 'datetime', 'visit_count'],
      dtype='object')

In [38]:
intakes_latest.columns

Index(['datetime', 'animal_id', 'name', 'intake_type', 'intake_condition',
       'animal_type', 'sex_upon_intake', 'age_upon_intake', 'breed', 'color'],
      dtype='object')

# Merge outcomes and intakes:

In [39]:
merged_df = pd.merge(
    outcomes_latest,
    intakes_latest,
    on="animal_id",
    how="inner",
    suffixes=("_outcome", "_intake")
)

In [40]:
print(merged_df)

       animal_id name_outcome     outcome_type animal_type_outcome  \
0        A006100        Scamp  Return to Owner                 Dog   
1        A047759         Oreo         Transfer                 Dog   
2        A134067       Bandit  Return to Owner                 Dog   
3        A141142       Bettie  Return to Owner                 Dog   
4        A163459        Sasha  Return to Owner                 Dog   
...          ...          ...              ...                 ...   
145598   A929599          NaN         Transfer                 Cat   
145599   A929600          NaN         Transfer                 Cat   
145600   A929602          NaN         Transfer                 Cat   
145601   A929631        Fiona       Euthanasia                 Dog   
145602   A929710          NaN       Euthanasia                 Cat   

       sex_upon_outcome age_upon_outcome                  breed_outcome  \
0         Neutered Male         10 years           Spinone Italiano Mix   
1        

In [41]:
merged_df.columns

Index(['animal_id', 'name_outcome', 'outcome_type', 'animal_type_outcome',
       'sex_upon_outcome', 'age_upon_outcome', 'breed_outcome',
       'color_outcome', 'datetime_outcome', 'visit_count', 'datetime_intake',
       'name_intake', 'intake_type', 'intake_condition', 'animal_type_intake',
       'sex_upon_intake', 'age_upon_intake', 'breed_intake', 'color_intake'],
      dtype='object')

In [42]:
# merged_df[['animal_id', 'name_outcome', 'name_intake']] # I have checked all names match - We can drop the name from the outcomes table

In [43]:
# merged_df[['animal_id', 'animal_type_outcome', 'animal_type_intake']] # I have checked all animal_types match - We can drop the outcome one

In [44]:
# merged_df[['animal_id', 'sex_upon_outcome', 'sex_upon_intake']]
# Compare sex_upon_outcome vs sex_upon_intake (treat NaN == NaN as True)
sex_equal = merged_df['sex_upon_outcome'].fillna("MISSING") == merged_df['sex_upon_intake'].fillna("MISSING")
# Check overall result
print("All equal?", sex_equal.all())
# Count mismatches
print("Number of mismatches:", (~sex_equal).sum())
# See mismatched rows
mismatches = merged_df.loc[~sex_equal, ['animal_id', 'sex_upon_outcome', 'sex_upon_intake']]
print(mismatches.head(10))

# They are changing since at the time of intake some ar intact and later they get spayed.
# For now we can keep both of these variables and then later we can drop the sex_upon_outcome since we can't really use it for prediction

All equal? False
Number of mismatches: 63010
    animal_id sex_upon_outcome sex_upon_intake
607   A501076    Spayed Female   Intact Female
609   A501255    Neutered Male     Intact Male
639   A504813    Spayed Female   Intact Female
672   A510475    Neutered Male     Intact Male
774   A524634    Spayed Female   Intact Female
821   A530122    Spayed Female   Intact Female
862   A533106    Neutered Male     Intact Male
898   A536136    Spayed Female   Intact Female
912   A538342    Spayed Female   Intact Female
967   A544105    Neutered Male     Intact Male


In [45]:
# merged_df['sex_upon_intake'].value_counts()

In [46]:
# merged_df[['animal_id', 'age_upon_intake', 'age_upon_outcome']]

In [47]:
cols_to_drop = ['name_outcome', 'animal_type_outcome', 'breed_outcome', 'color_outcome']
merged_df = merged_df.drop(columns=cols_to_drop)
print(merged_df.columns)

Index(['animal_id', 'outcome_type', 'sex_upon_outcome', 'age_upon_outcome',
       'datetime_outcome', 'visit_count', 'datetime_intake', 'name_intake',
       'intake_type', 'intake_condition', 'animal_type_intake',
       'sex_upon_intake', 'age_upon_intake', 'breed_intake', 'color_intake'],
      dtype='object')


In [48]:
merged_df = merged_df.rename(columns={
    'name_intake': 'name',
    'animal_type_intake': 'animal_type',
    'breed_intake': 'breed',
    'color_intake': 'color'
})
print(merged_df.columns)

Index(['animal_id', 'outcome_type', 'sex_upon_outcome', 'age_upon_outcome',
       'datetime_outcome', 'visit_count', 'datetime_intake', 'name',
       'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake',
       'age_upon_intake', 'breed', 'color'],
      dtype='object')


In [49]:
final_order = [
    'animal_id', 'name', 'sex_upon_outcome', 'sex_upon_intake',
    'age_upon_outcome', 'age_upon_intake',
    'datetime_outcome', 'datetime_intake',
    'visit_count', 'intake_type', 'intake_condition',
    'animal_type', 'breed', 'color', 'outcome_type'
]

merged_df = merged_df[final_order]

print(merged_df.columns)

Index(['animal_id', 'name', 'sex_upon_outcome', 'sex_upon_intake',
       'age_upon_outcome', 'age_upon_intake', 'datetime_outcome',
       'datetime_intake', 'visit_count', 'intake_type', 'intake_condition',
       'animal_type', 'breed', 'color', 'outcome_type'],
      dtype='object')


In [50]:
# Keep only rows where age_at_intake <= age_at_outcome
merged_df = merged_df[merged_df['age_upon_intake'] <= merged_df['age_upon_outcome']]

# Now drop the outcome-only columns
merged_df = merged_df.drop(
    ['age_upon_outcome', 'datetime_outcome', 'sex_upon_outcome'],
    axis=1
).reset_index(drop=True)

In [51]:
print(merged_df.columns)
print(merged_df.shape)

Index(['animal_id', 'name', 'sex_upon_intake', 'age_upon_intake',
       'datetime_intake', 'visit_count', 'intake_type', 'intake_condition',
       'animal_type', 'breed', 'color', 'outcome_type'],
      dtype='object')
(135235, 12)


In [87]:
# Save merged_df to CSV
merged_df.to_csv("merged_outcomes_intakes.csv", index=False)